## PyTorch basics

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
a = torch.Tensor([4, 4, 6, 7, 7])

In [ ]:
a

In [ ]:
b = torch.LongTensor([4, 4, 6, 7, 7])

In [ ]:
b

In [ ]:
b * 2

In [ ]:
c = torch.Tensor([[1,2,3], [4,5,6.6]])

In [ ]:
c

In [ ]:
c.numpy()

In [ ]:
import numpy as np

In [ ]:
q = np.array([[1,2],[3,4],[5,6]])

In [ ]:
r = torch.Tensor(q)

In [ ]:
r

In [ ]:
torch.softmax(a, dim=0)

In [ ]:
a2 = a.unsqueeze(0)

In [ ]:
a2

In [ ]:
a2.shape, a.shape

In [ ]:
torch.softmax(a2, dim=0)

In [ ]:
torch.softmax(a, dim=0).sum()

In [ ]:
torch.softmax(c, dim=0)

In [ ]:
torch.softmax(c, dim=1)[0].sum()

## Load and process the bean data

In [ ]:
from scipy.io import arff
import pandas as pd
import numpy as np 
import matplotlib.pyplot as pyplot

In [ ]:
arffdata = arff.loadarff("/home/xsayas@GU.GU.SE/scratch/lt2222-v21-resources/DryBeanDataset/Dry_Bean_Dataset.arff")

In [ ]:
beandf = pd.DataFrame(arffdata[0])

In [ ]:
len(beandf)

In [ ]:
beandf[10000:10020]

In [ ]:
arffdata[1]

In [ ]:
classes = list(beandf['Class'].unique())
beandf['classid'] = beandf['Class'].map(lambda x: classes.index(x))

In [ ]:
classes, beandf[['Class','classid']].iloc[np.r_[100:120, 10000:10020]]

In [ ]:
beangroups = beandf.groupby("Class", group_keys=False)

In [ ]:
beangroups

In [ ]:
testdf = beangroups.apply(lambda x: x.sample(frac=0.2))

In [ ]:
len(beangroups)

In [ ]:
testdf[100:120]

In [ ]:
testdf.index

In [ ]:
testdf["Class"].unique()

In [ ]:
traindf = beandf.drop(testdf.index, axis=0)

In [ ]:
len(traindf)

In [ ]:
len(testdf)

In [ ]:
10890 + 2721, len(beandf)

In [ ]:
train_X = traindf.drop(['Class','classid'], axis=1)

In [ ]:
train_X.columns

In [ ]:
train_y = traindf['classid']

In [ ]:
train_y

In [ ]:
test_X = testdf.drop(['Class', 'classid'], axis=1)

In [ ]:
test_y = testdf['classid']
test_y

## PyTorch modeling

In [ ]:
train_X_t = torch.Tensor(train_X.to_numpy())
train_y_t = torch.LongTensor(train_y.to_numpy())
test_X_t = torch.Tensor(test_X.to_numpy())
test_y_t = torch.LongTensor(test_y.to_numpy())

In [ ]:
train_X_t, train_X_t.shape

In [ ]:
train_y_t, train_y_t.shape

In [ ]:
len(train_X.columns)

In [ ]:
train_X_t = train_X_t.unsqueeze(0)
train_X_t, train_X_t.shape

In [ ]:
lin1 = nn.Linear(16, 10)

In [ ]:
outputs = lin1(train_X_t)

In [ ]:
outputs, outputs.shape

In [ ]:
relu = nn.ReLU()

In [ ]:
outputs = relu(outputs)
outputs, outputs.shape

In [ ]:
lin2 = nn.Linear(10, 7)

In [ ]:
outputs = lin2(outputs)
outputs, outputs.shape

In [ ]:
softmax = nn.LogSoftmax(dim=1)

In [ ]:
outputs = softmax(outputs)
outputs, outputs.shape

In [ ]:
criterion = nn.NLLLoss()

In [ ]:
outputs = outputs.squeeze(0)

In [ ]:
criterion(outputs, train_y_t)

In [ ]:
class BeanModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lin1 = nn.Linear(16, 300)
        self.tanh = nn.Tanh()
        self.lin2 = nn.Linear(300, 7)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.tanh(x)
        x = self.lin2(x)
        return self.softmax(x)

In [ ]:
train_y_t[[1, 5, 10000]]

In [ ]:
import random
# (not real batching as it doesn't guarantee we'll cover all the data in an epoch)
def batches(trainx, trainy, percent):
    while True:
        indices = list(range(len(trainy)))
        indices = random.choices(indices, k=int(percent * len(indices)))
        yield trainx[:, indices], trainy[indices]

In [ ]:
b = batches(train_X_t, train_y_t, 0.25)
next(b)

In [ ]:
tx, ty = next(b)
tx.shape, ty.shape, train_y_t.shape

In [ ]:
model = BeanModel()
model(tx)

In [ ]:
def train(tx, ty, epochs, batchpercent):
    m = BeanModel()
    
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(m.parameters(), lr=0.1)
    batcher = batches(tx, ty, batchpercent)
    
    # we crudely compensate for the fact that we aren't doing real batching by expanding the number of epochs.
    for i in range(int(epochs/batchpercent)):
        X, y = next(batcher)
        
        optimizer.zero_grad()
        outputs = m(X)
        loss = criterion(outputs.squeeze(0), y)
        loss.backward()
        optimizer.step()
        print("Epoch {} has loss {}.".format(i, loss))
        
    return m

In [ ]:
model = train(train_X_t, train_y_t, 200, 0.25)

## Evaluation

In [ ]:
model.eval()

In [ ]:
outputs = model(test_X_t.unsqueeze(0))

In [ ]:
outputs, outputs.shape

In [ ]:
predictions = pd.Series(outputs.squeeze(0).argmax(dim=1).numpy())

In [ ]:
predictions

In [ ]:
test_y

In [ ]:
predictions.index = test_y.index
predictions

In [ ]:
accuracy = len(test_y[predictions == test_y]) / len(test_y)

In [ ]:
accuracy